In [1]:
%matplotlib inline
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import scipy.stats as stats
import sklearn.linear_model as linear_model
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import KFold
from IPython.display import HTML, display
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from bubbly.bubbly import bubbleplot 


pd.options.display.max_rows = 1000
pd.options.display.max_columns = 20

In [2]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as sp
import plotly.io as pio
pio.templates
from plotly.subplots import make_subplots

In [3]:
import warnings

# 경고 무시하기
warnings.filterwarnings("ignore")
# unicode minus를 사용하지 않기 위한 설정 (minus 깨짐현상 방지)
plt.rcParams['axes.unicode_minus'] = False
# 한글 텍스트를 제대로 표시하기 위해 'font.family' 사용
plt.rcParams['font.family'] = 'AppleGothic'

In [4]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [5]:
# 최종 전세 파일 오픈 
housing_price = pd.read_csv('./data_1/강남구전세데이터_찐마지막.csv', encoding='utf-8')
housing_price.head()

,AD_Code,Admin_District,Area,YearBuilt,Type,Floor,Address,Cont_Date,Age,Latitude,Longitude,Star_Cnt,Michelin_Cnt,Mc_cnt,Bigstore_cnt,InterestRate,Market_cnt,Subway_cnt,Kinder_cnt,y
0,0,개포동,44.0,1989,단독다가구,3.0,서울특별시 강남구 논현로4길,2022-06-01,34,37.473722,127.052725,0,0,0,0,4.04,0,0,2,7500
1,0,개포동,44.0,1991,단독다가구,3.0,서울특별시 강남구 논현로4길,2022-10-05,32,37.473722,127.052725,0,0,0,0,4.82,0,0,2,14333
2,0,개포동,44.0,1991,단독다가구,3.0,서울특별시 강남구 논현로4길,2022-07-07,32,37.473722,127.052725,0,0,0,0,4.16,0,0,2,17115
3,0,개포동,44.0,1991,단독다가구,3.0,서울특별시 강남구 논현로4길,2022-07-07,32,37.473722,127.052725,0,0,0,0,4.16,0,0,2,17640
4,0,개포동,44.0,1989,단독다가구,3.0,서울특별시 강남구 논현로4길,2022-06-01,34,37.473722,127.052725,0,0,0,0,4.04,0,0,2,7500


In [6]:
df = housing_price

In [7]:
# Cont_Date 문자열 형태 -> datetime 으로 변경
df['Cont_Date'] = df['Cont_Date'].apply(lambda x: pd.to_datetime(str(x), format='%Y-%m-%d'))

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87361 entries, 0 to 87360
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   AD_Code         87361 non-null  int64         
 1   Admin_District  87361 non-null  object        
 2   Area            87361 non-null  float64       
 3   YearBuilt       87361 non-null  int64         
 4   Type            87361 non-null  object        
 5   Floor           87361 non-null  float64       
 6   Address         87361 non-null  object        
 7   Cont_Date       87361 non-null  datetime64[ns]
 8   Age             87361 non-null  int64         
 9   Latitude        87361 non-null  float64       
 10  Longitude       87361 non-null  float64       
 11  Star_Cnt        87361 non-null  int64         
 12  Michelin_Cnt    87361 non-null  int64         
 13  Mc_cnt          87361 non-null  int64         
 14  Bigstore_cnt    87361 non-null  int64         
 15  In

In [10]:
df['AD_Code'].value_counts()
df['Admin_District'].value_counts()
df['Type'].value_counts()

9     14741
2     13290
0     10842
3      7445
1      7022
4      6350
11     5586
8      4578
12     4458
6      4141
13     3962
5      2556
7      1624
10      766
Name: AD_Code, dtype: int64

역삼동     14741
대치동     13290
개포동     10842
도곡동      7445
논현동      7022
삼성동      6350
일원동      5586
압구정동     4578
자곡동      4458
수서동      4141
청담동      3962
세곡동      2556
신사동      1624
율현동       766
Name: Admin_District, dtype: int64

아파트      53963
연립다세대    15781
단독다가구     9472
오피스텔      8145
Name: Type, dtype: int64

In [11]:
## 상준님 top 100 아파트 큰 금액 별로 리스트 뽑음 

# 필터링 조건
top_100_apartments = (df['Admin_District'].isin(['도곡동', '압구정동', '대치동', '청담동', '삼성동'])) & (df['Type'] == '아파트')

# 조건에 맞는 데이터 추출 후 보증금(y)을 기준으로 내림차순 정렬 후 상위 100개 선택
top_100_expensive_apartments = df[top_100_apartments].nlargest(100, 'y')

# 결과 출력
print(top_100_expensive_apartments[['Admin_District', 'Address', 'y']])

      Admin_District                 Address        y
61207            청담동    서울특별시 강남구 압구정로79길 88  1100000
61206            청담동    서울특별시 강남구 압구정로79길 88   930000
61203            청담동    서울특별시 강남구 도산대로99길 68   710000
63674            청담동    서울특별시 강남구 압구정로75길 27   700000
64271            청담동    서울특별시 강남구 압구정로71길 28   620000
61204            청담동    서울특별시 강남구 도산대로99길 68   600000
61205            청담동    서울특별시 강남구 도산대로99길 68   500000
61486            청담동   서울특별시 강남구 영동대로142길 19   500000
63901            도곡동  서울특별시 강남구 남부순환로373길 23   500000
33439            도곡동     서울특별시 강남구 언주로30길 56   480000
61485            청담동   서울특별시 강남구 영동대로142길 19   480000
34076            도곡동     서울특별시 강남구 언주로30길 26   460000
34081            도곡동     서울특별시 강남구 언주로30길 26   460000
34067            도곡동     서울특별시 강남구 언주로30길 26   450000
36408            삼성동      서울특별시 강남구 봉은사로 645   450000
37796            삼성동       서울특별시 강남구 삼성로 636   450000
61288            청담동     서울특별시 강남구 학동로97길 31   450000
61289            청담동     서울특

In [13]:
# 상위 100개 정보를 CSV 파일로 저장
top_100_expensive_apartments[['Admin_District', 'Address', 'y']].to_csv('./data_1/top_100_apartments.csv', index=False)